In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import mannwhitneyu, chi2_contingency, kruskal

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
%matplotlib inline

## 1. Load Geographic Data

In [ ]:
# Load all relevant data
features = pd.read_csv("../data/processed/graph_structural_features.csv")
clusters = pd.read_csv("../data/processed/cluster_assignments_kmeans.csv")
summation = pd.read_csv("../data/processed/summation_test_results.csv")

# Load provenance from CSV
hierarchy = pd.read_csv("../data/processed/cord_hierarchy.csv")
provenance = hierarchy[['KHIPU_ID', 'PROVENANCE']].drop_duplicates()

# Merge
data = features.merge(clusters, on='khipu_id').merge(
    summation, on='khipu_id', how='left'
).merge(
    provenance, left_on='khipu_id', right_on='KHIPU_ID', how='left'
)

data['PROVENANCE'] = data['PROVENANCE'].fillna('Unknown')
data = data[data['PROVENANCE'].str.strip() != '']  # Remove empty strings

# Get major provenances (n >= 10)
prov_counts = data['PROVENANCE'].value_counts()
major_provs = prov_counts[prov_counts >= 10].index.tolist()

num_khipus = len(data)
num_major = len(major_provs)

print(f"✓ Loaded {num_khipus} khipus")
print(f"✓ Major provenances (n≥10): {num_major}")
print("\nTop 10 provenances:")
display(prov_counts.head(10))

## 2. Summation Rates by Provenance

In [ ]:
# Calculate summation statistics by provenance
prov_summation = []
for prov in major_provs:
    prov_data = data[data['PROVENANCE'] == prov]
    prov_summation.append({
        'Provenance': prov,
        'Count': len(prov_data),
        'Summation Rate (%)': prov_data['has_pendant_summation'].mean() * 100,
        'Avg Match Rate (%)': prov_data['pendant_match_rate'].mean() * 100,
        'White Boundary Rate (%)': prov_data['has_white_boundaries'].mean() * 100
    })

summation_df = pd.DataFrame(prov_summation).sort_values('Summation Rate (%)', ascending=False)

print("\nSummation Patterns by Provenance:")
display(summation_df)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Summation rate
ax1.barh(summation_df['Provenance'], summation_df['Summation Rate (%)'],
         color=plt.cm.viridis(summation_df['Summation Rate (%)'] / summation_df['Summation Rate (%)'].max()))
ax1.set_xlabel('Summation Rate (%)', fontsize=12, fontweight='bold')
ax1.set_title('Khipus with Summation Patterns by Provenance', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

for idx, row in summation_df.iterrows():
    ax1.text(row['Summation Rate (%)'] + 1, idx, f"{row['Summation Rate (%)']:.1f}%",
            va='center', fontweight='bold', fontsize=9)

# Match rate
ax2.barh(summation_df['Provenance'], summation_df['Avg Match Rate (%)'],
         color=plt.cm.plasma(summation_df['Avg Match Rate (%)'] / summation_df['Avg Match Rate (%)'].max()))
ax2.set_xlabel('Average Match Rate (%)', fontsize=12, fontweight='bold')
ax2.set_title('Summation Accuracy by Provenance', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical test
print("\nKruskal-Wallis Test (summation rate across provenances):")
groups = [data[data['PROVENANCE'] == prov]['has_pendant_summation'].dropna() for prov in major_provs]
stat, pval = kruskal(*groups)
print(f"H-statistic: {stat:.3f}, p-value: {pval:.4f}")
print(f"Interpretation: {'SIGNIFICANT' if pval < 0.05 else 'NOT SIGNIFICANT'} regional variation")

## 3. Structural Features by Provenance

In [ ]:
# Calculate structural statistics
prov_structure = []
for prov in major_provs:
    prov_data = data[data['PROVENANCE'] == prov]
    prov_structure.append({
        'Provenance': prov,
        'Avg Size': prov_data['num_nodes'].mean(),
        'Avg Depth': prov_data['depth'].mean(),
        'Avg Branching': prov_data['avg_branching'].mean(),
        'Numeric Coverage (%)': prov_data['has_numeric'].mean() * 100
    })

structure_df = pd.DataFrame(prov_structure)

print("\nStructural Features by Provenance:")
display(structure_df.round(2))

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

features_to_plot = [
    ('Avg Size', 'Average Khipu Size (nodes)', plt.cm.coolwarm),
    ('Avg Depth', 'Average Hierarchy Depth', plt.cm.viridis),
    ('Avg Branching', 'Average Branching Factor', plt.cm.plasma),
    ('Numeric Coverage (%)', 'Numeric Coverage (%)', plt.cm.cividis)
]

for idx, (feature, title, cmap) in enumerate(features_to_plot):
    ax = axes[idx // 2, idx % 2]
    sorted_df = structure_df.sort_values(feature, ascending=False)
    
    bars = ax.barh(sorted_df['Provenance'], sorted_df[feature],
                   color=cmap(sorted_df[feature] / sorted_df[feature].max()))
    
    ax.set_xlabel(feature, fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    for bar, val in zip(bars, sorted_df[feature]):
        ax.text(val + val*0.02, bar.get_y() + bar.get_height()/2,
               f'{val:.1f}', va='center', fontweight='bold', fontsize=9)

plt.suptitle('Structural Diversity Across Provenances', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Interactive Provenance Comparison

In [ ]:
# Select two provenances to compare
prov_a = widgets.Dropdown(
    options=sorted(major_provs),
    value='Incahuasi',
    description='Provenance A:'
)

prov_b = widgets.Dropdown(
    options=sorted(major_provs),
    value='Pachacamac',
    description='Provenance B:'
)

def compare_provenances(prov_a_val, prov_b_val):
    data_a = data[data['PROVENANCE'] == prov_a_val]
    data_b = data[data['PROVENANCE'] == prov_b_val]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle(f'{prov_a_val} (n={len(data_a)}) vs {prov_b_val} (n={len(data_b)})',
                fontsize=16, fontweight='bold')
    
    features = [
        ('num_nodes', 'Size (nodes)'),
        ('depth', 'Hierarchy Depth'),
        ('avg_branching', 'Avg Branching'),
        ('has_numeric', 'Numeric Coverage'),
        ('pendant_match_rate', 'Summation Match Rate'),
        ('num_white_boundaries', 'White Boundaries')
    ]
    
    for idx, (feature, label) in enumerate(features):
        ax = axes[idx // 3, idx % 3]
        
        data_list = [data_a[feature].dropna(), data_b[feature].dropna()]
        _ = ax.violinplot(data_list, positions=[1, 2], showmeans=True, showmedians=True)
        
        ax.set_xticks([1, 2])
        ax.set_xticklabels([prov_a_val[:10], prov_b_val[:10]], rotation=15)
        ax.set_ylabel(label, fontsize=11)
        ax.grid(True, alpha=0.3)
        
        mean_a = data_a[feature].mean()
        mean_b = data_b[feature].mean()
        diff_pct = ((mean_a - mean_b) / mean_b * 100) if mean_b != 0 else 0
        ax.set_title(f'{mean_a:.2f} vs {mean_b:.2f} ({diff_pct:+.1f}%)', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Statistical comparison
    print("\nStatistical Tests (Mann-Whitney U):")
    for feature, label in features:
        _, pval = mannwhitneyu(data_a[feature].dropna(), data_b[feature].dropna())
        sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else "ns"
        print(f"  {label}: p={pval:.4f} {sig}")
    
    # Cluster distribution
    print("\nCluster Distribution:")
    cluster_a = data_a['cluster'].value_counts().sort_index()
    cluster_b = data_b['cluster'].value_counts().sort_index()
    comparison = pd.DataFrame({
        prov_a_val: cluster_a,
        prov_b_val: cluster_b
    }).fillna(0).astype(int)
    display(comparison)

output = widgets.interactive_output(
    compare_provenances,
    {'prov_a_val': prov_a, 'prov_b_val': prov_b}
)

display(widgets.HBox([prov_a, prov_b]))
display(output)

## 5. Cluster × Provenance Enrichment

In [ ]:
# Calculate enrichment: observed / expected ratio
contingency = pd.crosstab(data['cluster'], data['PROVENANCE'])
print("\nCluster × Provenance Contingency Table:")
display(contingency)

# Chi-square test
chi2, pval, dof, expected = chi2_contingency(contingency)
print(f"\nChi-square test: χ²={chi2:.2f}, p={pval:.4f}, df={dof}")
print(f"Interpretation: {'SIGNIFICANT' if pval < 0.05 else 'NOT SIGNIFICANT'} association\n")

# Enrichment ratios
enrichment = contingency.copy()
for i in range(len(contingency)):
    for j in range(len(contingency.columns)):
        observed = contingency.iloc[i, j]
        exp = expected[i, j]
        enrichment.iloc[i, j] = observed / exp if exp > 0 else 0

print("Enrichment Ratios (observed/expected):")
display(enrichment.round(2))

# Heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(enrichment, annot=True, fmt='.2f', cmap='RdYlGn', center=1.0, 
            cbar_kws={'label': 'Enrichment (obs/exp)'})
plt.title('Cluster × Provenance Enrichment', fontsize=14, fontweight='bold')
plt.xlabel('Provenance', fontsize=12)
plt.ylabel('Cluster', fontsize=12)
plt.tight_layout()
plt.show()

print("\nNotable Enrichments (ratio > 1.5):")
for i in range(len(enrichment)):
    for j in range(len(enrichment.columns)):
        ratio = enrichment.iloc[i, j]
        if ratio > 1.5 and contingency.iloc[i, j] >= 5:
            cluster = enrichment.index[i]
            prov = enrichment.columns[j]
            count = contingency.iloc[i, j]
            print(f"  Cluster {cluster} × {prov}: {ratio:.2f}x (n={count})")

## 6. Geographic Insights Summary

In [ ]:
# Generate summary insights
print("\n" + "="*70)
print("GEOGRAPHIC PATTERN SUMMARY")
print("="*70)

# Top summation provenance
top_summation = summation_df.iloc[0]
print(f"\n1. HIGHEST SUMMATION: {top_summation['Provenance']}")
print(f"   Rate: {top_summation['Summation Rate (%)']:.1f}%")
print(f"   Avg Match: {top_summation['Avg Match Rate (%)']:.1f}%")

# Largest khipus
largest_prov = structure_df.sort_values('Avg Size', ascending=False).iloc[0]
print(f"\n2. LARGEST KHIPUS: {largest_prov['Provenance']}")
print(f"   Avg Size: {largest_prov['Avg Size']:.1f} nodes")
print(f"   Avg Depth: {largest_prov['Avg Depth']:.2f} levels")

# Deepest hierarchy
deepest_prov = structure_df.sort_values('Avg Depth', ascending=False).iloc[0]
print(f"\n3. DEEPEST HIERARCHY: {deepest_prov['Provenance']}")
print(f"   Avg Depth: {deepest_prov['Avg Depth']:.2f} levels")

# Most branching
branching_prov = structure_df.sort_values('Avg Branching', ascending=False).iloc[0]
print(f"\n4. HIGHEST BRANCHING: {branching_prov['Provenance']}")
print(f"   Avg Branching: {branching_prov['Avg Branching']:.2f}")

print("\n" + "="*70)